In [2]:
import os
import pickle

from skimage.io import imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.svm import SVC

In [3]:
input_dir = 'clf-data'
categories = ['empty', 'not_empty']

In [4]:
data= []
labels=[]

In [5]:
for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path)
        img = resize(img, (15, 15))
        data.append(img.flatten()) #flattened into a long array (uni-dim)
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, shuffle=True, stratify=labels)

In [15]:
classifier = SVC()
# we take 12 different variations of the classifier
parameters = [{'gamma':[0.01,0.001,0.0001], 'C': [1,10,100,1000]}]

In [16]:
#exhaustive search over the specified param values for the classifier
grid_search = GridSearchCV(classifier, parameters) 
grid_search.fit(X_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [17]:
#performance testing
best_estimator = grid_search.best_estimator_
y_prediction = best_estimator.predict(X_test)

In [18]:
# Calculating accuracy
accuracy = accuracy_score(y_test, y_prediction)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.9994


In [19]:
# Creating and printing confusion matrix
conf_matrix = confusion_matrix(y_test, y_prediction)
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculating and printing F1 score
f1 = f1_score(y_test, y_prediction)
print(f'F1 Score: {f1:.4f}')

Confusion Matrix:
[[914   0]
 [  1 840]]
F1 Score: 0.9994


In [20]:
#Saving the model for future use
pickle.dump(best_estimator, open('./model.p', 'wb'))